In [1]:
import base58
import binascii
import ecdsa
import hashlib
import json
import os
import subprocess
import time

def dSHA256(data):
    hash_1 = hashlib.sha256(data).digest()
    hash_2 = hashlib.sha256(hash_1).digest()
    return hash_2

def hash160(s):
    '''sha256 followed by ripemd160'''
    return hashlib.new('ripemd160', hashlib.sha256(s).digest()).digest()

def privkey_to_pubkey(privkey):
    signing_key = ecdsa.SigningKey.from_string(privkey, curve=ecdsa.SECP256k1) # Don't forget to specify the curve
    verifying_key = signing_key.get_verifying_key()

    # Use this code block if the address you gave corresponds to the compressed public key
    x_cor = bytes.fromhex(verifying_key.to_string().hex())[:32] # The first 32 bytes are the x coordinate
    y_cor = bytes.fromhex(verifying_key.to_string().hex())[32:] # The last 32 bytes are the y coordinate
    if int.from_bytes(y_cor, byteorder="big", signed=True) % 2 == 0: # We need to turn the y_cor into a number.
        public_key = bytes.fromhex("02" + x_cor.hex())
    else:
        public_key = bytes.fromhex("03" + x_cor.hex())
    return public_key

In [2]:
# Functions related to generating bitcoin addresses

def encode_base58(s):
    BASE58_ALPHABET = '123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz'
    count = 0
    for c in s:  
        if c == 0:
            count += 1
        else:
            break
    num = int.from_bytes(s, 'big')
    prefix = '1' * count
    result = ''
    while num > 0:  
        num, mod = divmod(num, 58)
        result = BASE58_ALPHABET[mod] + result
    return prefix + result

def encode_base58_checksum(b):
    return encode_base58(b + dSHA256(b)[:4])

# https://en.bitcoin.it/wiki/Wallet_import_format
def privkey_to_wif(privkey, compressed_pubkey, testnet):
    if testnet:
        prefix = b"\xEF"
    else:
        prefix = b"\x80"
    
    # if the privkey will correspond to a compressed public key
    if compressed_pubkey: 
        extended = prefix + privkey + b"\x01"
    else:
        extended = prefix + privkey 
        
    extendedchecksum = extended + dSHA256(extended)[:4]
    wif = encode_base58(extendedchecksum)
    
    return wif

# https://learnmeabitcoin.com/guide/wif
def wif_to_privkey(private_key_WIF):
    private_key_full = base58.b58decode(private_key_WIF)
    
    # If the WIF encoding includes the optional "01" byte for compressed privKey,
    # do not include it in the final output.
    if len(private_key_full) == 38:
        private_key = private_key_full[1:-5] 
        print("compressed pubkey")
    else:
        private_key = private_key_full[1:-4] 
        print("not compressed pubkey")
    return private_key


def pk_to_p2pkh(compressed, testnet):
    '''Returns the address string'''
    pk_hash = hash160(compressed)
    if testnet:
        prefix = b'\x6f'
    else:
        prefix = b'\x00'
    return encode_base58_checksum(prefix + pk_hash)

def pk_to_p2sh_p2wpkh(compressed, testnet):
    pk_hash = hash160(compressed)
    redeemScript = bytes.fromhex(f"0014{pk_hash.hex()}")
    rs_hash = hash160(redeemScript)
    if testnet:
        prefix = b"\xc4"
    else:
        prefix = b"\x05"
    return encode_base58_checksum(prefix + rs_hash)

def wif_to_addresses(wif):
#     wif = "cUy9rC6wteKizfu1fgP2abKUWTkJxjqKp2fba91FkU332CFHo6ix"
    privkey = wif_to_privkey(wif)
    public_key = privkey_to_pubkey(privkey)
    p2pkh_address = pk_to_p2pkh(public_key, testnet = True)
    p2sh_p2wpkh_address = pk_to_p2sh_p2wpkh(public_key, testnet = True)

    print("WIF Private key: " + wif)
    print("Private key: " + privkey.hex())
    print("Public key: " + public_key.hex())
    print("Public key hash: " + hash160(public_key).hex())
    print("Address: " + p2pkh_address)
    print("Address: " + p2sh_p2wpkh_address)
    
def privkey_to_addresses(privkey):
    # privkey = bytes.fromhex("AF933A6C602069F1CBC85990DF087714D7E86DF0D0E48398B7D8953E1F03534A")
    public_key = privkey_to_pubkey(privkey)
    p2pkh_address = pk_to_p2pkh(public_key, testnet = True)
    p2sh_p2wpkh_address = pk_to_p2sh_p2wpkh(public_key, testnet = True)

    print("Private key: " + privkey.hex())
    print("Public key: " + public_key.hex())
    print("Public key hash: " + hash160(public_key).hex())
    print("Address: " + p2pkh_address)
    print("Address: " + p2sh_p2wpkh_address)

# Bitcoin core regtest tx tester
This notebook will start up bitcoind in regtest mode, create transactions using python scripts, and broadcast them to validate that they work. Four transactions are tested here:<br><br>

- tx1 initial tx to fund customer. 1 output 
    - Send 11.3 btc to a P2SH-P2WPKH address. This will be used to fund the customer.
- tx2 funding tx with change. 2 outputs
    - 10.2 btc to an escrow (2-of-2 multisig) 
    - 1 btc to a change address
- tx3 cust close. 2 outputs
    - 2 btc to a merch_close_pubkey output
    - 8.1 btc to p2wsh with two ways to spend from it:
        - to_self_delay to cust_close_pk (cust-claim-tx)
        - revocation preimage & merch_disp signature (merch-dispute-tx)

Both ways of spending from cust-close are tested below

Generate all the priv/public keys

In [3]:
funding_privkey_hex = "8911111111111111111111111111111111111111111111111111111111111111"
funding_pubkey_hex = privkey_to_pubkey(bytes.fromhex(funding_privkey_hex)).hex()

merch_privkey_hex = "3911111111111111111111111111111111111111111111111111111111111111"
merch_pubkey_hex = privkey_to_pubkey(bytes.fromhex(merch_privkey_hex)).hex()

cust_privkey_hex = "7911111111111111111111111111111111111111111111111111111111111111"
cust_pubkey_hex = privkey_to_pubkey(bytes.fromhex(cust_privkey_hex)).hex()

cust_close_privkey_hex = "7711111111111111111111111111111111111111111111111111111111111111"
cust_close_pubkey_hex = privkey_to_pubkey(bytes.fromhex(cust_close_privkey_hex)).hex()

change_privkey_hex = "6911111111111111111111111111111111111111111111111111111111111111"
change_pubkey_hex = privkey_to_pubkey(bytes.fromhex(change_privkey_hex)).hex()

merch_close_privkey_hex = "3711111111111111111111111111111111111111111111111111111111111111"
merch_close_pubkey_hex = privkey_to_pubkey(bytes.fromhex(merch_close_privkey_hex)).hex()

merch_disp_privkey_hex = "3111111111111111111111111111111111111111111111111111111111111111"
merch_disp_pubkey_hex = privkey_to_pubkey(bytes.fromhex(merch_disp_privkey_hex)).hex()

revocation_secret_hex = "4011111111111111111111111111111111111111111111111111111111111111"
revocation_lock_hex = hash160(bytes.fromhex(revocation_secret_hex)).hex()

Start up regtest mode, delete any history so we are starting from scratch.
Mine 101 blocks

In [4]:
# Make sure bitcoind is not already running
os.system("bitcoin-cli -regtest stop")
time.sleep(1.5) 

# Delete any previous files to restart regtest
os.system("rm -rfv $HOME/Library/Application\ Support/Bitcoin/regtest/")

# start up bitcoind in regtest mode
os.system("bitcoind -regtest -daemon")
time.sleep(1.5)

# generate 101 blocks so we can fund transactions
os.system("bitcoin-cli -regtest generate 101")
blockcount = subprocess.getoutput("bitcoin-cli -regtest getblockcount")

print("blockcount: " + str(blockcount))

blockcount: 101


Generate base58 address for the escrow funder

In [5]:
# Generate p2sh-p2wpkh address to fund the escrow funder
privkey = bytes.fromhex(funding_privkey_hex)
public_key = privkey_to_pubkey(privkey)
p2sh_p2wpkh_address = pk_to_p2sh_p2wpkh(public_key, testnet = True)

# print("Private key: " + privkey.hex())
# print("Public key: " + public_key.hex())
print("Address: " + p2sh_p2wpkh_address)

Address: 2N1jKbzrUkf583t9mM9ePN3rkqdCwryKcf9


Send btc to the escrow funder

In [6]:
txid_1 = subprocess.getoutput("bitcoin-cli -regtest sendtoaddress " + p2sh_p2wpkh_address + " 11.3")
print(txid_1)

f9ce2c4091906a5e8041db4e4da359f580ff875daf2bea318520ff856a5d33dc


In [7]:
# Find which output index the btc was sent to
raw_tx = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + txid_1)
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + raw_tx)
d = json.loads(decoded)
# print(decoded)

if d["vout"][0]["scriptPubKey"]["addresses"][0] == p2sh_p2wpkh_address:
    index = 0
else:
    index = 1
print("index: " + str(index))

index: 0


In [8]:
os.system("bitcoin-cli -regtest generate 1");

Generate raw escrow funding transaction

In [9]:
raw_escrow_tx = subprocess.getoutput("python funding_tx_with_change.py" 
                        + " --funding_privkey " + funding_privkey_hex
                        + " --txid " + txid_1
                        + " --index " + str(index)
                        + " --amount_btc " + "11.3"
                        + " --merch_pubkey " + merch_pubkey_hex
                        + " --cust_pubkey " + cust_pubkey_hex
                        + " --change_pubkey " + change_pubkey_hex
                        + " --escrow_btc " + "10.2"
                        + " --change_btc " + "1")
print(raw_escrow_tx)

02000000000101dc335d6a85ff208531ea2baf5d87ff80f559a34d4edb41805e6a9091402ccef90000000017160014fc5cc2b7bdcb852b225434d133769a551486950affffffff02fff6cb3c00000000220020666c6bfa88ba97c90cb04c7038d56b5854e71a4dd174d79b1260c822a14f791e00e1f50500000000160014f3a8a4335c0ef84806d93315e56965f13d522e5f024730440220195e52dd28d94ebe83eeda77868fead9a2f4927271b1b433fa171019068160d102204f02d33d4993dd2962d07af8eaf48878d2b69974bc6e7b48515e011606efe238012103e2aa89cce89e9b2d6f09b20a2096226328f114a4ca62e6ea416b4d7c4573086e00000000


Broadcast escrow funding transaction

In [10]:
escrow_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + raw_escrow_tx + " true")
# "true" flag means we are okay with absurdly high tx fee
print(escrow_txid)

eb00fb921db2ae8ec254a1258334989fa6d31240f9d094b8f6bba3732cf624f9


In [11]:
raw_escrow_tx1 = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + escrow_txid)
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + raw_escrow_tx1)
# print(decoded)

In [12]:
os.system("bitcoin-cli -regtest generate 1");

Create transaction spending from the change output of the escrow transaction

In [13]:
raw_cust_close_tx = subprocess.getoutput("python cust_close_from_escrow.py"        
                + " --cust_privkey " + cust_privkey_hex 
                + " --merch_privkey " + merch_privkey_hex      
                + " --merch_close_pubkey " + merch_close_pubkey_hex   
                + " --merch_disp_pubkey " + merch_disp_pubkey_hex     
                + " --cust_close_pubkey " + cust_close_pubkey_hex     
                + " --revocation_lock " + revocation_lock_hex      
                + " --to_self_delay " + "05cf" # number of blocks to delay to-self output       
                + " --txid " + escrow_txid
                + " --index " + "0"
                + " --amount_btc " + "10.2"
                + " --script_output_btc " + "8.1"
                + " --merch_output_btc " + "2")
print(raw_cust_close_tx)

decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + raw_cust_close_tx)
# print(decoded)

02000000000101f924f62c73a3bbf6b894d0f94012d3a69f98348325a154c28eaeb21d92fb00eb0000000000ffffffff03809e47300000000022002024fdfddacb774716169995d839b3eb07809a5133c4d4a6254615b113f2a4e82e00c2eb0b00000000160014d4354803d10e77eccfc3bf06c152ae694d05d3810000000000000000376a35bf1ad41a96ff238d0b6fee50c8e2cd5bd757ab8803195e272df2310ded35f9958fd0c2847bf73b5b429a716c005d465009bd768641040047304402207e99a781cdce5805591b4de5f58e3f6eb46882f17ae193a7803c8593186013c302203500e9a76595331d8ec350aec37b9715bfc00dcdb4a35a16ddb19aea27adb4510147304402206818b9b88a49dc04872883906eca6b7959a9c39c5b3fd9b9e2f7aed50eadcb1d022043db03189604c479a0e11f87b48da66263b164bd49df15870ca7ac1e0c4b9caf0147522102f3d17ca1ac6dcf42b0297a71abb87f79dfa2c66278cbb99c1437e6570643ce902103fc43b44cd953c7b92726ebefe482a272538c7e40fdcde5994a62841525afa8d752ae00000000


In [14]:
cust_close_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + raw_cust_close_tx + " true")
print(cust_close_txid)

82152c40bd80aee4f7b09b9131e189448734ca7d0e66423180c7098c55d51532


In [15]:
cust_close_tx = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + cust_close_txid)
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + cust_close_tx)
print(decoded)

{
  "txid": "82152c40bd80aee4f7b09b9131e189448734ca7d0e66423180c7098c55d51532",
  "hash": "24f17e10cad008413149206b8bd9bd005f9f006b46bfbf9a1c97ca952f8a67e8",
  "version": 2,
  "size": 409,
  "vsize": 244,
  "weight": 976,
  "locktime": 0,
  "vin": [
    {
      "txid": "eb00fb921db2ae8ec254a1258334989fa6d31240f9d094b8f6bba3732cf624f9",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "",
        "304402207e99a781cdce5805591b4de5f58e3f6eb46882f17ae193a7803c8593186013c302203500e9a76595331d8ec350aec37b9715bfc00dcdb4a35a16ddb19aea27adb45101",
        "304402206818b9b88a49dc04872883906eca6b7959a9c39c5b3fd9b9e2f7aed50eadcb1d022043db03189604c479a0e11f87b48da66263b164bd49df15870ca7ac1e0c4b9caf01",
        "522102f3d17ca1ac6dcf42b0297a71abb87f79dfa2c66278cbb99c1437e6570643ce902103fc43b44cd953c7b92726ebefe482a272538c7e40fdcde5994a62841525afa8d752ae"
      ],
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value": 

## Cust-claim from cust-close is valid after to_self_delay

In [16]:
raw_cust_claim_tx = subprocess.getoutput("python cust_claim.py"   
                + " --cust_close_privkey " + cust_close_privkey_hex 
                + " --output_pubkey " + "024596d7b33733c28101dbc6c85901dffaed0cdac63ab0b2ea141217d1990ad4b1"    
                + " --merch_disp_pubkey " + merch_disp_pubkey_hex   
                + " --revocation_lock " + revocation_lock_hex     
                + " --to_self_delay " + "05cf" # number of blocks to delay to-self output         
                + " --txid " + cust_close_txid
                + " --index " + "0"
                + " --amount_btc " + "8.1"
                + " --output_btc " + "8")
print(raw_cust_claim_tx)


decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + raw_cust_claim_tx)
# print(decoded)

020000000001013215d5558c09c7803142660e7dca34874489e131919bb0f7e4ae80bd402c15820000000000cf050000010008af2f00000000160014b7cfef435e3701fdedb7a11164ae44d561698bb9034830450221008588bc52827ae8993a9dd1ead60281fbf7eee158bc7e95a2db17f198173e346702203dc833d39f4dade36ba5a4e3c5fcc7ddf00ecf569e2d7889577bdf036e124bb701006463a914bf1ad41a96ff238d0b6fee50c8e2cd5bd757ab8888210253be79afe84fd9342c1f52024379b6da6299ea98844aee23838e8e678a765f7c6702cf05b2752103195e272df2310ded35f9958fd0c2847bf73b5b429a716c005d465009bd76864168ac00000000


In [17]:
os.system("bitcoin-cli -regtest generate 1486");

Only 1486 blocks mined, one less than the delay_to_self of 1487 blocks

In [18]:
cust_claim_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + raw_cust_claim_tx + " true")
print(cust_claim_txid)

error code: -26
error message:
non-BIP68-final (code 64)


Generate 1 more block and try again

In [19]:
os.system("bitcoin-cli -regtest generate 1");

In [20]:
# cust_claim_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + raw_cust_claim_tx + " true")
# print(cust_claim_txid)

In [21]:
# cust_claim_tx = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + cust_claim_txid)
# decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + cust_claim_tx)
# print(decoded)

## Merch-dispute-tx

In [22]:
raw_merch_dispute_tx = subprocess.getoutput("python merch_dispute.py"   
                + " --merch_disp_privkey " + merch_disp_privkey_hex 
                + " --revocation_secret " + revocation_secret_hex 
                + " --cust_close_pubkey " + cust_close_pubkey_hex   
                + " --output_pubkey " + "024596d7b33733c28101dbc6c85901dffaed0cdac63ab0b2ea141217d1990ad4b1"    
                + " --to_self_delay " + "05cf" # number of blocks to delay to-self output         
                + " --txid " + cust_close_txid
                + " --index " + "0"
                + " --amount_btc " + "8.1"
                + " --output_btc " + "8")
print(raw_merch_dispute_tx)


# decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + raw_merch_dispute_tx)
# print(decoded)

020000000001013215d5558c09c7803142660e7dca34874489e131919bb0f7e4ae80bd402c15820000000000ffffffff010008af2f00000000160014b7cfef435e3701fdedb7a11164ae44d561698bb9044730440220665f9b0d0d966452b99d9b057eaf56b27e17c3bd7d0d1a0e81c48a66e2267a95022015c25af3f88cd19a0c8a5045e967dce244c5957f4d7e6541f05709a1f1a43df50120401111111111111111111111111111111111111111111111111111111111111101016463a914bf1ad41a96ff238d0b6fee50c8e2cd5bd757ab8888210253be79afe84fd9342c1f52024379b6da6299ea98844aee23838e8e678a765f7c6702cf05b2752103195e272df2310ded35f9958fd0c2847bf73b5b429a716c005d465009bd76864168ac00000000


In [23]:
merch_dispute_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + raw_merch_dispute_tx + " true")
print(merch_dispute_txid)

e38a0aabf2998823f5a7e1696df4bf2737cd582526bc4dbe0e7f9e4c4344a17b


In [24]:
merch_dispute_tx = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + merch_dispute_txid)
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + merch_dispute_tx)
print(decoded)

{
  "txid": "e38a0aabf2998823f5a7e1696df4bf2737cd582526bc4dbe0e7f9e4c4344a17b",
  "hash": "bb35dda2a717d0098915b4ab8707d43ce22bd8d3aefc2ccfa199ab41ba1b7d75",
  "version": 2,
  "size": 293,
  "vsize": 135,
  "weight": 539,
  "locktime": 0,
  "vin": [
    {
      "txid": "82152c40bd80aee4f7b09b9131e189448734ca7d0e66423180c7098c55d51532",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "30440220665f9b0d0d966452b99d9b057eaf56b27e17c3bd7d0d1a0e81c48a66e2267a95022015c25af3f88cd19a0c8a5045e967dce244c5957f4d7e6541f05709a1f1a43df501",
        "4011111111111111111111111111111111111111111111111111111111111111",
        "01",
        "63a914bf1ad41a96ff238d0b6fee50c8e2cd5bd757ab8888210253be79afe84fd9342c1f52024379b6da6299ea98844aee23838e8e678a765f7c6702cf05b2752103195e272df2310ded35f9958fd0c2847bf73b5b429a716c005d465009bd76864168ac"
      ],
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value": 8.00000000,
      

In [25]:
os.system("bitcoin-cli -regtest stop")
time.sleep(1.5) 